In [ ]:
import heapq as hq
from math import sqrt
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import shuffle 

class Event:
    def __init__(self, time, action, TMName, passenger): #TM = transport medium
        self.time = time
        self.action = action
        self.tm = TMName
        self.p = passenger
    def __lt__(self, other):
        return (self.time, self.tm, self.p) < (other.time, other.tm, other.p)
    def __le__(self, other):
        return (self.time, self.tm, self.p) <= (other.time,  other.tm, other.p)

class Passenger:               
    def __init__(self, Target):
        self.target = Target
        self.change = 9 #change the vehicle, 8 = number of nodes
        self.time = 0
        self.tm = 'dummy'
    def __lt__(self, other):
        return (self.time, self.tm, self.target, self.change) < (other.time, other.tm, other.target, other.change)
    def __le__(self, other):
        return (self.time, self.tm, self.target, self.change) <= (other.time, other.tm, other.target, other.change)

class Vehicle:
    def __init__(self, Name, Color, Capacity, Itinerary, TravelTimes):
        self.name = Name
        self.color = Color
        self.capacity = Capacity
        self.itinerary = Itinerary
        self.travel_times = TravelTimes
        self.n_passengers = []
        self.stop = 0

def shuffle0(passengers):
    for i in range(len(passengers)):
        x = [j for j in range(len(passengers[i]))]
        shuffle(x)
        passengers[i] = [passengers[i][k] for k in x]

def calc_pic(grah, ListColor, ListSize):
    nx.draw(graph, nx.get_node_attributes(graph,'pos'), with_labels = True, node_color = ListColor, node_size = ListSize)

# cap = cut of possible ways which a vehicle can pass (documentation)
def cap(amount):
    temp = []
    if len(amount) == 0 or len(amount)== 1:
        return amount
    elif len(amount) == 2:
        for i in amount[0]:
            if i in amount[1]:
                temp += [i]
    else:
        return print('Error')
    return [temp]
#default='#ecf2e3'
default='#cad3b8' #nice color

In [ ]:
# Lösung für Passagier als Klasse
def solve(eventq, passengers, tms, graph):
    TIME = []
    Loop = []
    v = [len([idx for idx, passenger in enumerate(passengers[n]) if passenger.target == n]) == 0 for n in range(len(passengers))]
    if not (v == [True]*len(passengers)):
        ErrorMessage = 'Bei Passengers/Netz sind an den Knoten {} Passagiere, die auf demselben Ziel untergebracht sind.'.format([idx for idx, boolean in enumerate(v) if boolean == False])
        raise Exception(ErrorMessage)
    
    if not (len(graph.node)) == (len(passengers)):
        ErrorMessage = 'Die Graphiken werden fehlerhaft berechnet.\n'
        if len(graph.node) < len(passengers):
            ErrorMessage += 'Es sind mehr Einträge in Passengers ({}) als Knoten im Graph ({})'.format(len(passengers),len(graph.node)) + '\n'
            ErrorMessage += 'Löschen/Fügen Sie ({}) Einträge/Knoten im Passengers/Graph /ein'.format(abs(len(passengers) - len(graph.node)))
        else:
            ErrorMessage += 'Es sind weniger Einträge in Passengers ({}) als Knoten im Graph ({})'.format(len(passengers),len(graph.node)) + '\n'
            ErrorMessage += 'Fügen/Löschen Sie ({}) Einträge/Knoten im Passengers/Graph /ein'.format(abs(len(passengers) - len(graph.node)))
        raise Exception(ErrorMessage)
    
    #if not [(len(tms[i].itinerary == len(tms[i].travel_times)) for i in range(len(tms))] == [True]*len(tms):
    #    raise Exception('In der Liste TMS sind itinerary und travel_times nicht gleich groß.')
    
    cycle = 0
    while True:
        print('\n Durchläuft die While-Schleife zum {}. Mal'.format(cycle))
       
        ''' # Ausgabe der Queue mit Checks
        m='Länge der Queue: {} Queue: ['.format(len(eventq))
        for el in eventq:
            p = el.p
            m += '[Time: {}.{}, Aktion: {}, TMName:{}.{}]; '.format(el.time, el.time==p.time, el.action, el.tm, el.tm==p.tm)
        print(m+']')
        #'''
        
        # Abbruchbedingung
        print([len(tm.n_passengers) for tm in tms])
        print([len(node) for node in passengers])
        Vehicle = [idx for idx, gondola in enumerate(tms) if gondola.name == eventq[0].tm]
        vehicle = tms[Vehicle[0]]
        Loop += [{'Cycle': cycle,'PassengerVehicle': [len(tm.n_passengers) for tm in tms],'PassengerNode': [len(node) for node in passengers], 'Time': eventq[0].time, 'Aktion': eventq[0].action, 'Vehicle': vehicle}]
        if (([len(node) == 0 for node in passengers] == [True]*len(passengers)) and 
           ([len(tm.n_passengers) == 0 for tm in tms] == [True]*len(tms))):
            print('Ende')
            TMstat = [{'Name': tm.name,'Stopps': tm.stop, 'Länge der insgesamt zurückgelegten Strecke': len(tm.itinerary)} for tm in tms]
            return TMstat, TIME, Loop
        
        event = hq.heappop(eventq)
        
        Gondolas = [idx for idx, gondola in enumerate(tms) if gondola.name == event.tm]
        if not Gondolas:
            raise Exception('Error')
        if len(Gondolas)>1:
            raise Exception('Error')
        tm = tms[Gondolas[0]]

        if tm.name != 'dummy':
            itinerary_len = len(tm.itinerary)
            current_node = tm.itinerary[tm.stop % itinerary_len]
            cn_number = [idx for idx, station in enumerate(tm.itinerary) if station == current_node][0] #Current Node Number
        
            next_node = tm.itinerary[(tm.stop + 1) % itinerary_len]
            nn_number = [idx for idx, station in enumerate(tm.itinerary) if station == next_node][0]
            print('Fahrzeug {} ist am Knoten {}, der nächste Halt {}, die aktuelle Zeit: {}'.format(tm.name,ListNodes[current_node], ListNodes[next_node], event.time))
            print('und {} Stopp zum {}. Mal, Runden: {}, Rest: {}'.format(tm.name, tm.stop, int(tm.stop/itinerary_len),tm.stop % itinerary_len))
            stop = tm.stop
        else:
            current_node = random.randint(0,len(passengers)-1)
            while current_node == event.p.target:
                current_node = random.randint(0,len(passengers)-1)
            print('Fahrzeug {} ist am Knoten {}, der nächste Halt ist zufällig, die aktuelle Zeit: {}'.format(tm.name, ListNodes[current_node], event.time))
        
        if event.action == 'load':
            print('Fahrzeug {} load'.format(tm.name))
            print('Fahrzeug {} hat {} Plätze frei'.format(tm.name,tm.capacity - len(tm.n_passengers)))
            dummy = event.p
            change = []
            print('Es warten {} am Knoten {}.'.format(len(passengers[current_node]), ListNodes[current_node]))
            # Konventionelles Auffüllen
            while len(tm.n_passengers) < tm.capacity:
                if not current_node < len(passengers):
                    raise Exception('Die Länge von passengers {} ist <= current_node {}'.format(len(passengers),current_node))
                waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger.target in tm.itinerary]
                if not waiting:
                    break
                passenger = passengers[current_node].pop(waiting[0])
                passenger.tm = tm.name
                
                exit_node = passenger.target
                en_number = [idx for idx, station in enumerate(tm.itinerary) if station == exit_node][0]
                
                ListExit = []    
                for i in range((en_number-cn_number)% itinerary_len):
                    ListExit += [ tm.travel_times[(cn_number + i) % itinerary_len] ]
                ExitTime = event.time - 1
                for i in ListExit:
                    ExitTime += (i)
                #print('ExitTime: {}, ListExit: {}'.format(ExitTime,ListExit))
                
                passenger.time = ExitTime
                hq.heappush(tm.n_passengers, ExitTime)
                tm.n_passengers.sort()
                event_new = Event(ExitTime,'exit', tm.name, passenger)
                if not event_new.time == passenger.time:
                    raise Exception('Fehler beim Setzen der Event Time')
                if not event_new.tm == passenger.tm:
                    raise Exception('Fehler beim Setzen des Namens des Beförderungsmittels')
                hq.heappush(eventq, event_new)
            print('Berechne, wer zukünftig umsteigen soll')
            Zaehler = 0
            while len(tm.n_passengers) < tm.capacity:
                print('Zweite while in load {}'.format(Zaehler))
                if not current_node < len(passengers):
                    raise Exception('Die Länge von passengers {} ist <= current_node {}'.format(len(passengers),current_node))
                waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger.target in tm.itinerary]
                change = passengers[current_node][:]
                counter = 0
                while len(waiting) != 0:
                    temp = waiting.pop(0) - counter
                    change.pop(temp)
                    counter += 1
                change = [idx for idx, passenger in enumerate(change) if passenger in passengers[current_node]]
                
                if not passengers[current_node]:
                    break
                if not change:
                    break
                
                break2while = False
                for changeindex in range(len(change)):
                    p = passengers[current_node][change[0]]
                    protect = [tm.name for tm in tms if p.target in tm.itinerary and current_node in tm.itinerary]
                
                    SortHelpToChange = []
                    for htc in helpToChange: #htc = help to change
                        if current_node in htc:
                            SortHelpToChange += [htc]
                    for htc in helpToChange:
                        if p.target in htc:
                            SortHelpToChange += [htc]
                    print('Protect: ',len(protect) != 0)
                    print('p.target {}, current_node {}'.format(p.target, current_node))
                    print('SortHelpToChange: ', SortHelpToChange)
                    # Anfang der Schutzfunktion
                    if not protect:
                        path = []
                        path0 = []
                        for i in [[i for i in htc if i != current_node and i != p.target] for htc in SortHelpToChange]:
                            path0 += i
                        path1 = [current_node]
                        if len(SortHelpToChange) ==2:
                            path1 += [i for i in SortHelpToChange[1] if i != p.target]
                        path2 = []
                        for i in [ [i for i in htc] for htc in SortHelpToChange if current_node in htc]:
                            path2 += i
                        print('path0', path0)
                        print('path1', path1)
                        print('path2', path2)
                        
                        PATH0 = [[tm.name for tm in tms if node in tm.itinerary] for node in path0]
                        PATH1 = [[tm.name for tm in tms if node in tm.itinerary] for node in path1]
                        PATH2 = [[tm.name for tm in tms if node in tm.itinerary] for node in path2]
                        print('PATH0 ',PATH0)
                        print('PATH1 ',PATH1)
                        print('PATH2 ',PATH2)
                        #
                        # Hack gilt nur für dieses Szenario
                        print(cap(PATH0))
                        print(cap(PATH1))
                        print(cap(PATH2))
                        
                        print([len(i) != 0 for i in cap(PATH0)])
                        print([len(i) != 0 for i in cap(PATH1)])
                        print([len(i) != 0 for i in cap(PATH2)])
                        
                        if not ([len(i) != 0 for i in cap(PATH0)] == [True]):
                            if not ([len(i) != 0 for i in cap(PATH1)] == [True]):
                                path = path2
                                print('path2')
                            else:
                                if tm.name in [tm.name for tm in tms[:2]]:
                                    path = path1
                                    print('path1')
                                else:
                                    print('break')
                                    break2while = True
                        else:
                            path = path0
                            print('path0')
                        print('der gewält: {}'.format(path))
                        break
                    else:
                        print('changeindex: {};len von change {}'.format(changeindex,len(change)))
                
                '''
                p = passengers[current_node][change[0]]
                protect = [tm.name for tm in tms if p.target in tm.itinerary and current_node in tm.itinerary]
                
                SortHelpToChange = []
                for htc in helpToChange: #htc = help to change
                    if current_node in htc:
                        SortHelpToChange += [htc]
                for htc in helpToChange:
                    if p.target in htc:
                        SortHelpToChange += [htc]
                print('Protect: ',len(protect) != 0)
                #print('p.target {}, current_node {}'.format(p.target, current_node))
                #print('SortHelpToChange: ', SortHelpToChange)
                # Anfang der Schutzfunktion
                if not protect:
                    path = []
                    path0 = []
                    for i in [[i for i in htc if i != current_node and i != p.target] for htc in SortHelpToChange]:
                        path0 += i
                    path1 = [current_node]
                    if len(SortHelpToChange) ==2:
                        path1 += [i for i in SortHelpToChange[1] if i != p.target]
                    path2 = []
                    for i in [ [i for i in htc] for htc in SortHelpToChange if current_node in htc]:
                        path2 += i
                    #print('path0', path0)
                    #print('path1', path1)
                    #print('path2', path2)
                    
                    PATH0 = [[tm.name for tm in tms if node in tm.itinerary] for node in path0]
                    PATH1 = [[tm.name for tm in tms if node in tm.itinerary] for node in path1]
                    PATH2 = [[tm.name for tm in tms if node in tm.itinerary] for node in path2]
                    #print('PATH0 ',PATH0)
                    #print('PATH1 ',PATH1)
                    #print('PATH2 ',PATH2)
                    #
                    # Hack gilt nur für dieses Szenario
                    #print(cap(PATH0))
                    #print(cap(PATH1))
                    #print(cap(PATH2))
                    
                    #print([len(i) != 0 for i in cap(PATH0)])
                    #print([len(i) != 0 for i in cap(PATH1)])
                    #print([len(i) != 0 for i in cap(PATH2)])
                    
                    if not ([len(i) != 0 for i in cap(PATH0)] == [True]):
                        if not ([len(i) != 0 for i in cap(PATH1)] == [True]):
                            path = path2
                        else:
                            if tm.name in [tm.name for tm in tms[:2]]:
                                path = path1
                            else:
                                break
                    else:
                        path = path0
                    #print('der gewält: {}'.format(path))
                else:
                    break
                '''
                # Ende der Schutzfunktion
                print('changeindex: {};len von change {}'.format(changeindex,len(change)))
                if (changeindex == len(change) - 1) or break2while:
                    break
                passenger = passengers[current_node].pop(change.pop(changeindex))
                if not p == passenger: raise Exception('Gedankenfehler')
                
                # Das Setzen des Umstiegs ist vermutlich vom Hack abhängig
                print(path)
                if len(path) == 1:
                    passenger.change = path[0]
                elif len(path) == 2:
                    l = [i for i in path if current_node != i]
                    passenger.change = l[0]
                else:
                    raise Exception('Gedankenfehler')
                
                passenger.tm = tm.name
                exit_node = passenger.change
                #print('exit_node ',exit_node)
                if not [idx for idx, station in enumerate(tm.itinerary) if station == exit_node]:
                    break
                en_number = [idx for idx, station in enumerate(tm.itinerary) if station == exit_node][0]
                
                ListExit = []    
                for i in range((en_number-cn_number)% itinerary_len):
                    ListExit += [ tm.travel_times[(cn_number + i) % itinerary_len] ]
                ExitTime = event.time - 1
                for i in ListExit:
                    ExitTime += (i)
                #print('ExitTime: {}, ListExit: {}'.format(ExitTime,ListExit))
                
                passenger.time = ExitTime
                hq.heappush(tm.n_passengers, ExitTime)
                tm.n_passengers.sort()
                event_new = Event(ExitTime,'exit', tm.name, passenger)
                if not event_new.time == passenger.time:
                    raise Exception('Fehler beim Setzen der Event Time')
                if not event_new.tm == passenger.tm:
                    raise Exception('Fehler beim Setzen des Namens des Beförderungsmittels')
                hq.heappush(eventq, event_new)
                Zaehler += 1
                
            Time = event.time + tm.travel_times[nn_number]
            dummy.time = Time
            hq.heappush(eventq, Event(Time, 'load', tm.name, dummy))
            tm.stop += 1
            print('Es warten noch {} Fahrgäste an Knoten {}'.format(len(passengers[current_node]),ListNodes[current_node]))
        
        # Generate events for exiting passengers
        elif event.action == 'exit':
            info = [{'Zeit': True}]
            passenger = event.p
            ExitTime = hq.heappop(tm.n_passengers)
            print('PassengerStation:{}, PassengerTime:{}, steigt aus, EventTime {}'.format(passenger.target, passenger.time,event.time))
            print('Ist EventTime == PassengerTime: {}'.format(passenger.time == event.time))
            
            if (passenger.time != event.time) and (tm.name != 'dummy'):
                info = [{'Zeit': False}]
                print(passenger)
                gondola = passenger.tm
                Gondolas = [idx for idx, gondola in enumerate(tms) if gondola.name == passenger.tm]
                if not Gondolas:
                    raise Exception('Error')
                if len(Gondolas)>1:
                    raise Exception('Error')
                tm0 = tms[Gondolas[0]]
                print('Passenger TM: {} CList: {}'.format(gondola, tm0.n_passengers))
                print('Event TM: {} CList: {}'.format(tm.name, tm.n_passengers))
                
            print('Passagier am richtigen Ort: {}'.format([passenger.target == current_node, passenger.change == current_node]))
            if not [passenger.target == current_node,passenger.change == current_node] == [False]*2:
                    info += [{'Ort': True}]
            if ([passenger.target == current_node,passenger.change == current_node] == [False]*2):
                info += [{'Ort': False}]
                if tm.name != 'dummy':
                    print(passenger)
                    gondola = passenger.tm
                    print(gondola, tm.name)
                    print('Passagier mit Ziel: {} wurde falsch berechnet.'.format(passenger.target))
                passengers[current_node] += [passenger]
                shuffle0(passengers)
            if not passenger.change == current_node:
                info += [{'Change': False}]
            if passenger.change == current_node:
                info += [{'Change': True}]
                passengers[current_node] += [passenger]
                shuffle0(passengers)
                print('Passagier mit Ziel: {} steigt um.'.format(passenger.target))
                passenger.change = 8
            print('Es sind {} Passagiere in der Bahn'.format(len(tm.n_passengers)))
            if tm.name != 'dummy':
                TIME += [{'Info': info ,'EventTime': event.time, 'PassengerTime': passenger.time}]
        else:
            raise Exception('Unknown event action')
        #eventq.sort()
        cycle += 1

In [ ]:
graph = nx.Graph()

graph.add_node('Hbf', pos=(10,10))  #0 #0
graph.add_node('INF', pos=(10,30))  #1 #1
graph.add_node('TBA0', pos=(20,35)) # #2
graph.add_node('Phil', pos=(30,30))  #2 #3
graph.add_node('Uni', pos=(40,20))  #3 #4
graph.add_node('Bism', pos=(30,10))  #4 #5
graph.add_node('Rohr', pos=(30,0))  #5 #6
graph.add_node('Hand', pos=(15,40))  #6 #7
graph.add_node('Zieg', pos=(50,30))  #7 #8
 #8

ListNodes = list(graph.node)
#Rundfahrt 
graph.add_edge(ListNodes[0],ListNodes[1])
graph.add_edge(ListNodes[1],ListNodes[0])
graph.add_edge(ListNodes[1],ListNodes[2])
graph.add_edge(ListNodes[2],ListNodes[1])
graph.add_edge(ListNodes[2],ListNodes[3])
graph.add_edge(ListNodes[3],ListNodes[2])
graph.add_edge(ListNodes[3],ListNodes[4])
graph.add_edge(ListNodes[4],ListNodes[3])
graph.add_edge(ListNodes[4],ListNodes[5])
graph.add_edge(ListNodes[5],ListNodes[4])
graph.add_edge(ListNodes[5],ListNodes[0])
graph.add_edge(ListNodes[0],ListNodes[5])

#Zubringer
graph.add_edge(ListNodes[5],ListNodes[6])
graph.add_edge(ListNodes[6],ListNodes[5])
graph.add_edge(ListNodes[1],ListNodes[7])
graph.add_edge(ListNodes[7],ListNodes[1])
graph.add_edge(ListNodes[4],ListNodes[8])
graph.add_edge(ListNodes[8],ListNodes[4])

ListNodes = list(graph.node)
ListColor = [default]*len(ListNodes)
nx.draw(graph, nx.get_node_attributes(graph,'pos'), with_labels = True, node_color = ListColor, node_size = 500)

In [ ]:
#""" 74, 40, 40, 40
capa = 12
Color = ['#f0f000','#f000f0','#ff0000','#00ff00']
tms = []
for i in range(1):
    name = "cycle0%02d" % i
    tms += [Vehicle(name, Color[0], 37*capa, [5,4, 3, 2, 1, 0], [5,5, 5, 5, 5, 5])]
for i in range(1):
    name = "cycle1%02d" % i
    tms += [Vehicle(name, Color[0], 37*capa, [0, 1, 2, 3, 4,5], [5,5, 5, 5, 5, 5])]
for i in range(1):
    name = "zubinger0%02d" % i
    tms += [Vehicle(name, Color[1], 40*capa, [5,6], [5,5])]
for i in range(1):
    name = "zubinger1%02d" % i
    tms += [Vehicle(name, Color[2], 40*capa, [1,7], [5,5])]
for i in range(1):
    name = "zubinger2%02d" % i
    tms += [Vehicle(name, Color[3], 40*capa, [4,8], [5,5])]
#"""

arm0 = 18 #3600 # 5 # 8
arm1 = 75  #1500 # 1
arm2 = 10 #2000 # 1
arm3 = 15 #3000 # 1 # 3
passengers = [[]]*9
for node in range(len(passengers)):
    if node < 6:
        a = []
        for i in range(arm0//6):
            while True:
                target = random.randint(0,len(passengers)-1)
                if target != node:
                    break
            a += [Passenger(target)]
        passengers[node] = a
    elif 6 == node:
        a = []
        for i in range(arm1//2):
            while True:
                target = random.randint(0,len(passengers)-1)
                if target != node:
                    break
            a += [Passenger(target)]
        passengers[node] = a
    elif 7 == node:
        a = []
        for i in range(arm2//2):
            while True:
                target = random.randint(0,len(passengers)-1)
                if target != node:
                    break
            a += [Passenger(target)]
        passengers[node] = a
    elif 8 == node:
        a = []
        for i in range(arm3//2):
            while True:
                target = random.randint(0,len(passengers)-1)
                if target != node:
                    break
            a += [Passenger(target)]
        passengers[node] = a
    else:
        a = []
        passengers[node] = a#"""
shuffle0(passengers)

print(len(passengers))
print([len(i)for i in passengers])
helpToChange = [[5,6],[1,7],[4,8]]


# shuffle passengers

sums=0
for node in passengers:
    sums += len(node)
print(sums)

In [ ]:
#"""
eventq = []
for tm in tms:
    if tm.name != 'dummy':
        event = Event(0, 'load', tm.name, Passenger(_))
        event.p.tm = event.tm
        event.p.time = event.time
        hq.heappush(eventq, event)#"""


A = solve(eventq, passengers, tms, graph)

In [ ]:
NodeTime = [{'Node': index['PassengerNode'],'Time': index['Time']} for index in (A[2])]
NodeTime_sort0 =[[element for element in NodeTime if element['Time'] == time][0] for time in range(NodeTime[len(NodeTime) - 1]['Time']+1) if [element for element in NodeTime if element['Time'] == time]]

fig = plt
fig.figure(figsize=(13.5,7.5))
fig.plot([index['Time'] for index in NodeTime_sort0], [index['Node'] for index in NodeTime_sort0])
#fig.plot([index['Time'] for index in NodeTime_sort0], [sum(index['Node']) for index in NodeTime_sort0])
fig.title('')
fig.legend(ListNodes)
fig.xlabel('Zeit')
fig.ylabel('Passagiere')
fig.show()
#fig.savefig('nodes.svg')

In [ ]:
ListNodes = list(graph.node)
for index in A[2]:
    ListColor = [default]*len(ListNodes)
    ListSize = [sqrt(j)*100 for j in index['PassengerNode']]
    if index['Vehicle'].name != 'dummy':
        ininerary_len = len(index['Vehicle'].itinerary)
        current_node = index['Vehicle'].itinerary[index['Vehicle'].stop % ininerary_len]
        ListColor[current_node] = index['Vehicle'].color
    plt.figure(index['Cycle'], figsize=(13.5,7.5))
    calc_pic(graph, ListColor, ListSize)
    Text = "Schleifendurchlauf: {}".format(index['Cycle']) + "\n" + "Zeit: {}, Aktion: {}".format(index['Time'],index['Aktion'])
    plt.text(14, 15, Text, fontsize=20)
    #plt.show()
    plt.savefig("img/file%06d.png" % index['Cycle'])

#for i in {$(find . -name 'file*.svg')}; do   echo $i; inkscape -z $i -h 370 -w 550 -d 100 -e $i.png; done
cat *.png | ffmpeg -framerate 5 -f image2pipe -i - animation1.mkv